In [2]:
import tensorflow as tf
import nltk
from nltk.corpus import gutenberg

# word embedding
词向量模型，主要功能是用于表示

参考
1）cs224n 第二讲和第三讲
2）speech and language processing chapter 16 -- semantic with dense vectors
3）https://www.jiqizhixin.com/articles/2020-06-30-11 推荐系统 embedding 技术实践总结

# 文本表征的方法
文本表征的目的是将文本转化成机器学习算法可以处理的数字形式，换句话说就是将文本使用数学语言进行表示。
- 词袋模型 one-hot\tf-idf
- 主题模型，LSA\PLSA\LDA等
- 词向量模型 word2vec fastText glove
- 动态词向量模型 elmo BERT GPT

词向量模型将一个词表示成一个先对要短（1000以内的长度）以及稠密的向量（大部分值不为0）
词袋模型简单，但是存在维数灾难和语义鸿沟的问题
主题模型训练计算复杂度高
早期的词向量的研究源于语言模型，比附NNLM和RNNLM, 词向量是副产物。
**语言模型，就是用来计算一个词序列的概率的模型。**


分布式假设：相同上下文语境的词具有相似含义，并且由此引出了word2vec和fastText等。这类词向量中，虽然其本质仍然是语言模型，但其目标是生成词向量。

embdding技术尽管非常实用，但是依然存在一些问题。【3】比如
- 增量更新前后语义不变
- cover多个维度的特征
- 多模态向量融合
- 长尾数据资料少难以充分训练
- Embedding空间分布，影响模型泛化误差

fastText 相比于word2vec, 训练词向量会考虑subword。还可以进行文本分类

glove 的特点
glove相比于LSA， 采用Adagrad 对最小平方损失进行优化，可以算是一种经过优化的矩阵分解算法。

glove 相比于word2vec， 使用了全局语料，同时glove不能进行在线学习，因为其需要统计固定语料信息

glove的损失韩式最小平方损失函数，相比于word2vec的带权重的交叉熵，


elmo、GPT和bert 
这三个均为动态词向量。动态词向量的特点是在不同上下文环境下，词向量也会不相同。这就解决了一词多义问题。

elmo使用LSTM进行提取特征，使用双向语言模型
GPT和bert使用transformer来提取特征
bert使用双向语言模型

word2vec 有两种实现方式, CBOW 和 Skip-gram

CBOW 使用上下文背景词来预测目标词
Skip-gram使用目标词来预测背景词


为了加快word2vec的训练速度，使用了negative sampling 和 Hierarchical softmax.




In [5]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [3]:
data = gutenberg.words("austen-emma.txt")

In [6]:
word_count = dict()
for d in data:
    d = d.lower()
    word_count[d] = word_count.get(d, 0) + 1

In [12]:
word_list = [(k, v) for k, v in word_count.items()]
word_list.sort(key=lambda x: x[1])

In [16]:
TOP = 10
WINDOW = 2

In [14]:
# 筛选词
word2id = {"UNK": 0}

top_num = len(word_list)*10//100
stop_list = word_list[:top_num] + word_list[-top_num:]
stop_set = set([k for k, _ in stop_list])
word2id =  {k:i+1 for i, (k, _) in  enumerate(word_list[top_num:-top_num])}
word2id["UNK"] = 0

In [20]:
VOCAB_SIZE = len(word2id)

# 数据预处理
将文本中的词进行转换，转换成数字

In [25]:
data_len = len(data)
context_data = []
label_data = []
temp_windows = ["P"*WINDOW]
for i, d in enumerate(data):
    temp_windows.append(d)
    
    context = []
    label = []
    if len(temp_windows) == 2*WINDOW+1:
        for j, t in enumerate(temp_windows):
            if t == "P":
                continue
            if j == WINDOW:
                continue
            context.append(word2id.get(t, 0))
        label.append(word2id.get(temp_windows[WINDOW], 0))
        temp_windows.pop(0)
    
    if context:
        context_data.append(context)
        label_data.append(label)
            
    if i == data_len-1:
        context = []
        label = []
        for j in range(WINDOW):
            
            
            for k, t in enumerate(temp_windows[j:]):
                if k == WINDOW:
                    continue
                context.append(word2id.get(t, 0))
            
            label.append(word2id.get(temp_windows[j+WINDOW], 0))
            
            context_data.append(context)
            label_data.append(label)
            

# 使用one-hot编码

In [33]:
context_data = [tf.keras.utils.to_categorical(d).sum(axis=0) for d in context_data]

In [18]:

class CBOW(tf.keras.Model):
    
    def __init__(self, vocab_size, embed_size):
        super(CBOW, self).__init__()
        
        self.embed = tf.keras.layers.Dense(embed_size)
        self.out = tf.keras.layers.Dense(vocab_size, activation="softmax")
        
    
    def call(self, input_data):
        
        x = self.embed(input_data)
        logits = self.out(x)
        
        return logits

In [17]:
optimiser = tf.keras.optimizers.Adam()


Help on class Dense in module tensorflow.python.keras.layers.core:

class Dense(tensorflow.python.keras.engine.base_layer.Layer)
 |  Just your regular densely-connected NN layer.
 |  
 |  `Dense` implements the operation:
 |  `output = activation(dot(input, kernel) + bias)`
 |  where `activation` is the element-wise activation function
 |  passed as the `activation` argument, `kernel` is a weights matrix
 |  created by the layer, and `bias` is a bias vector created by the layer
 |  (only applicable if `use_bias` is `True`).
 |  
 |  Note: If the input to the layer has a rank greater than 2, then
 |  it is flattened prior to the initial dot product with `kernel`.
 |  
 |  Example:
 |  
 |  ```python
 |  # as first layer in a sequential model:
 |  model = Sequential()
 |  model.add(Dense(32, input_shape=(16,)))
 |  # now the model will take as input arrays of shape (*, 16)
 |  # and output arrays of shape (*, 32)
 |  
 |  # after the first layer, you don't need to specify
 |  # the size 